# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [2]:
df.drop('salary', axis=1, inplace=True) # лик
X = df.drop('salary_in_usd', axis=1)
y = df['salary_in_usd']

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

In [5]:
from sklearn.preprocessing import OneHotEncoder

def apply_ohe_to_features(X_train, X_test, X_val, categorical_features):
    X_train = X_train.copy()
    X_test = X_test.copy()
    X_val = X_val.copy()

    for feature in categorical_features:
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        ohe.fit(X_train[[feature]])

        feature_names = ohe.get_feature_names_out([feature])

        X_train_ohe = pd.DataFrame(ohe.transform(X_train[[feature]]), columns=feature_names, index=X_train.index)
        X_test_ohe = pd.DataFrame(ohe.transform(X_test[[feature]]), columns=feature_names, index=X_test.index)
        X_val_ohe = pd.DataFrame(ohe.transform(X_val[[feature]]), columns=feature_names, index=X_val.index)

        X_train = pd.concat([X_train.drop(columns=[feature]), X_train_ohe], axis=1)
        X_test = pd.concat([X_test.drop(columns=[feature]), X_test_ohe], axis=1)
        X_val = pd.concat([X_val.drop(columns=[feature]), X_val_ohe], axis=1)

    return X_train.reset_index(drop=True), X_test.reset_index(drop=True), X_val.reset_index(drop=True)

In [6]:
categorical_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'remote_ratio',
       'company_location', 'company_size']

X_train, X_test, X_val = apply_ohe_to_features(X_train, X_test, X_val, categorical_features)

In [7]:
lr = LinearRegression().fit(X_train, y_train)

In [16]:
mape_train_lr = mean_absolute_percentage_error(y_train, lr.predict(X_train))
mape_test_lr = mean_absolute_percentage_error(y_test, lr.predict(X_test))
rmse_train_lr = mean_squared_error(y_train, lr.predict(X_train))**.5
rmse_test_lr = mean_squared_error(y_test, lr.predict(X_test))**.5

print(f'MAPE on train set: {mape_train_lr:.2f}')
print(f'MAPE on test set: {mape_test_lr:.2f}')
print(f'RMSE on train set: {rmse_train_lr:.2f}')
print(f'RMSE on test set: {rmse_test_lr:.2f}')

MAPE on train set: 0.31
MAPE on test set: 0.44
RMSE on train set: 45607.36
RMSE on test set: 45014.82


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [9]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.15, 0.2],
    'n_estimators': [200, 300, 500, 700, 900, 1100],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.05, 0.1, 0.2, 0.3, 0.4],
    'reg_alpha': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'reg_lambda': [0.5, 1.0, 1.5, 2.0]
}

xgb = XGBRegressor(objective='reg:absoluteerror', random_state=123)
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_grid, scoring='neg_mean_absolute_percentage_error', cv=3, verbose=1, n_iter=2000, random_state=123, n_jobs=-1)
random_search.fit(X_val, y_val)

print('Лучшие параметры:', random_search.best_params_)
print('Лучшее значение ошибки:', -random_search.best_score_)

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits
Лучшие параметры: {'subsample': 1.0, 'reg_lambda': 0.5, 'reg_alpha': 0.2, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.2, 'gamma': 0.2, 'colsample_bytree': 0.7}
Лучшее значение ошибки: 0.3800499637921651


In [ ]:
import time
t = time.time()
xgb = XGBRegressor(**random_search.best_params_, objective='reg:absoluteerror', random_state=123).fit(X_train, y_train)
print(f'Время обучения: {time.time() - t:.2f} секунд')

t = time.time()
xgb.predict(X_test)
print(f'Время предсказания: {time.time() - t:.5f} секунд')

mape_train_xgb = mean_absolute_percentage_error(y_train, xgb.predict(X_train))
mape_test_xgb = mean_absolute_percentage_error(y_test, xgb.predict(X_test))
rmse_train_xgb = mean_squared_error(y_train, xgb.predict(X_train))**.5
rmse_test_xgb = mean_squared_error(y_test, xgb.predict(X_test))**.5

print(f'MAPE on train set XGB: {mape_train_xgb:.2f}')
print(f'MAPE on test set XGB: {mape_test_xgb:.2f}')
print(f'RMSE on train set XGB: {rmse_train_xgb:.2f}')
print(f'RMSE on test set XGB: {rmse_test_xgb:.2f}')

Время обучения: 0.55 секунд
Время предсказания: 0.00377 секунд
MAPE on train set: 0.25
MAPE on test set: 0.34
RMSE on train set: 44866.53
RMSE on test set: 43338.98


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
from catboost import CatBoostRegressor

catboost = CatBoostRegressor(loss_function='MAE', random_seed=123, verbose=0)

param_grid = {
    'depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.15, 0.2],
    'iterations': [200, 300, 500, 700, 900, 1100],
    'l2_leaf_reg': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bylevel': [0.6, 0.7, 0.8, 0.9, 1.0],
    'random_strength': [0, 0.05, 0.1, 0.2, 0.3, 0.4],
    'bagging_temperature': [0, 0.05, 0.1, 0.2, 0.3, 0.4],
    'border_count': [32, 50, 100, 200]
}

random_search_catboost = RandomizedSearchCV(estimator=catboost, param_distributions=param_grid, scoring='neg_mean_absolute_percentage_error', cv=3, verbose=0, n_iter=200, random_state=123, n_jobs=-1)
random_search_catboost.fit(X_val, y_val)

print('Лучшие параметры CatBoost:', random_search_catboost.best_params_)
print('Лучшее значение ошибки CatBoost:', -random_search_catboost.best_score_)

Лучшие параметры CatBoost: {'subsample': 0.7, 'random_strength': 0.4, 'learning_rate': 0.15, 'l2_leaf_reg': 1e-05, 'iterations': 200, 'depth': 3, 'colsample_bylevel': 0.8, 'border_count': 50, 'bagging_temperature': 0.3}
Лучшее значение ошибки CatBoost: 0.38144552380876395


In [18]:
t = time.time()
catboost = CatBoostRegressor(**random_search_catboost.best_params_, loss_function='MAE', random_seed=123, verbose=0).fit(X_train, y_train, eval_set=(X_val, y_val))
print(f'Время обучения CatBoost: {time.time() - t:.2f} секунд')

t = time.time()
catboost.predict(X_test)
print(f'Время предсказания CatBoost: {time.time() - t:.5f} секунд')

mape_train_catboost = mean_absolute_percentage_error(y_train, catboost.predict(X_train))
mape_test_catboost = mean_absolute_percentage_error(y_test, catboost.predict(X_test))
rmse_train_catboost = mean_squared_error(y_train, catboost.predict(X_train))**.5
rmse_test_catboost = mean_squared_error(y_test, catboost.predict(X_test))**.5

print(f'MAPE on train set catboost: {mape_train_catboost:.2f}')
print(f'MAPE on test set catboost: {mape_test_catboost:.2f}')
print(f'RMSE on train set catboost: {rmse_train_catboost:.2f}')
print(f'RMSE on test set catboost: {rmse_test_catboost:.2f}')

Время обучения CatBoost: 0.19 секунд
Время предсказания CatBoost: 0.00194 секунд
MAPE on train set catboost: 0.30
MAPE on test set catboost: 0.33
RMSE on train set catboost: 47420.92
RMSE on test set catboost: 42743.29


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [13]:
from catboost import Pool

train_pool = Pool(X.iloc[y_train.index], y_train, cat_features=categorical_features)
test_pool = Pool(X.iloc[y_test.index], y_test, cat_features=categorical_features)
val_pool = Pool(X.iloc[y_val.index], y_val, cat_features=categorical_features)

In [14]:
catboost_pool = CatBoostRegressor(loss_function='MAE', random_seed=123, verbose=0).fit(train_pool, eval_set=val_pool, early_stopping_rounds=600)

print(f'MAPE on train set CatBoost with Pool: {mean_absolute_percentage_error(y_train, catboost_pool.predict(train_pool)):.2f}')
print(f'MAPE on test set CatBoost with Pool: {mean_absolute_percentage_error(y_test, catboost_pool.predict(test_pool)):.2f}')
print(f'RMSE on train set CatBoost with Pool: {mean_squared_error(y_train, catboost_pool.predict(train_pool))**.5:.2f}')
print(f'RMSE on test set CatBoost with Pool: {mean_squared_error(y_test, catboost_pool.predict(test_pool))**.5:.2f}')

MAPE on train set CatBoost with Pool: 0.29
MAPE on test set CatBoost with Pool: 0.34
RMSE on train set CatBoost with Pool: 46754.18
RMSE on test set CatBoost with Pool: 42843.70


**Ответ:** Модель получилась схожего качества, но степень переобучения ниже. По времени примерно то же самое.

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [15]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

lgbm = LGBMRegressor(random_state=123, verbose=-1)

param_grid_lgbm = {
    'num_leaves': [15, 31, 63],
    'max_depth': [-1, 3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 300, 500],
    'min_child_samples': [5, 10, 20],
    'min_child_weight': [1e-3, 1e-2, 1e-1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0.01, 0.1, 1],
    'max_bin': [127, 255],
    'boosting_type': ['gbdt']
}

search = RandomizedSearchCV(estimator=lgbm, param_distributions=param_grid_lgbm, n_iter=50, cv=5, scoring='neg_mean_absolute_percentage_error', verbose=0, n_jobs=-1, random_state=123)

search.fit(X_train, y_train)

print("Лучшие параметры lgbm:", search.best_params_)
print("Лучшее значение MAPE:", -search.best_score_)

Лучшие параметры lgbm: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0, 'num_leaves': 15, 'n_estimators': 100, 'min_child_weight': 0.1, 'min_child_samples': 5, 'max_depth': 7, 'max_bin': 127, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'boosting_type': 'gbdt'}
Лучшее значение MAPE: 0.3541118328925458


In [19]:
t = time.time()
lgbm = LGBMRegressor(**search.best_params_, random_state=123, verbose=-1).fit(X_train, y_train)
print(f'Время обучения LGBM: {time.time() - t:.2f} секунд')

t = time.time()
lgbm.predict(X_test)
print(f'Время предсказания LGBM: {time.time() - t:.5f} секунд')

mape_train_lgbm = mean_absolute_percentage_error(y_train, lgbm.predict(X_train))
mape_test_lgbm = mean_absolute_percentage_error(y_test, lgbm.predict(X_test))
rmse_train_lgbm = mean_squared_error(y_train, lgbm.predict(X_train))**.5
rmse_test_lgbm = mean_squared_error(y_test, lgbm.predict(X_test))**.5

print(f'MAPE on train set lgbm: {mape_train_lgbm:.2f}')
print(f'MAPE on test set lgbm: {mape_test_lgbm:.2f}')
print(f'RMSE on train set lgbm: {rmse_train_lgbm:.2f}')
print(f'RMSE on test set lgbm: {rmse_test_lgbm:.2f}')

Время обучения LGBM: 0.29 секунд
Время предсказания LGBM: 0.00195 секунд
MAPE on train set lgbm: 0.32
MAPE on test set lgbm: 0.39
RMSE on train set lgbm: 45295.99
RMSE on test set lgbm: 42969.05


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [22]:
losses = pd.DataFrame(index=['Linear', 'XGBoost', 'CatBoost', 'LGBM'], 
                     columns=['MAPE_train', 'MAPE_test', 'RMSE_train', 'RMSE_test'],
                     data=[
                         [mape_train_lr, mape_test_lr, rmse_train_lr, rmse_test_lr],
                         [mape_train_xgb, mape_test_xgb, rmse_train_xgb, rmse_test_xgb],
                         [mape_train_catboost, mape_test_catboost, rmse_train_catboost, rmse_test_catboost],
                         [mape_train_lgbm, mape_test_lgbm, rmse_train_lgbm, rmse_test_lgbm]])
losses.round(2)

,MAPE_train,MAPE_test,RMSE_train,RMSE_test
Linear,0.31,0.44,45607.36,45014.82
XGBoost,0.25,0.34,44866.53,43338.98
CatBoost,0.30,0.33,47420.92,42743.29
LGBM,0.32,0.39,45295.99,42969.05


**Ответ:** Самые лучшие модели с точки зрения качества - `CatBoost` и `XGBoost`, но первый переобучился сильно меньше, можно сказать что вообще не переобучился. Самой худшей получилась линейна модель - самое низкое качество и сильно переобучение, зато проста в использовинии, быстрая, не требует подбора гиперпараметров. Метрики `RMSE` для каждой из моделей идентичны, мало что можно про них сказать. По скорости обучения и предсказаний, очевидно, выигрывает линейна модель, но среди бустинга наиболее быстрым будет `CatBoost` - в данном случае в полтора раза быстрее `LGBM` и в два раза быстрее `XGBoost`. Для сорость предсказаний для `CatBoost` и `LGBM` идентична, почти в два раза быстрее `XGBoost`. Все параметры которые есть хотя бы у двух моделей отличаются, например - subsample, max_depth, learning_rate, n_estimators. Это связано с осбенностями каждого алгоритма, а также из-за наличия других гиперпараметров.

In [25]:
pd.DataFrame({
    'XGBoost': random_search.best_params_,
    'CatBoost': random_search_catboost.best_params_,
    'LightGBM': search.best_params_
})

,XGBoost,CatBoost,LightGBM
subsample,1.0,0.70000,0.8
reg_lambda,0.5,NaN,1
reg_alpha,0.2,NaN,0
n_estimators,200.0,NaN,100
max_depth,6.0,NaN,7
learning_rate,0.2,0.15000,0.1
gamma,0.2,NaN,NaN
colsample_bytree,0.7,NaN,0.8
random_strength,NaN,0.40000,NaN
l2_leaf_reg,NaN,0.00001,NaN


# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [50]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [51]:
ratings = ratings.T

Выкиньте строку под названием `user`.

In [52]:
ratings.drop('user', axis=0, inplace=True)

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [138]:
ratings.fillna(0, inplace=True)
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
system of a down,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [165]:
from sklearn.cluster import KMeans

ratings.drop('cluster', axis=1, inplace=True, errors='ignore')  # для перезапусков
kmeans = KMeans(n_clusters=5, random_state=1).fit(ratings)
ratings['cluster'] = kmeans.labels_
ratings.head()

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,cluster
the beatles,0.000000,0.000000,0.000000,0.0,0.043529,0.000000,0.0,0.0,0.093398,0.017621,...,0.0,0.121169,0.038168,0.007939,0.017884,0.0,0.076923,0.0,0.000000,1
radiohead,0.020417,0.184962,0.000000,0.0,0.086281,0.006322,0.0,0.0,0.000000,0.019156,...,0.0,0.000000,0.000000,0.011187,0.000000,0.0,0.000000,0.0,0.000000,0
deathcab for cutie,0.000000,0.024561,0.028635,0.0,0.034590,0.000000,0.0,0.0,0.000000,0.013349,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.027893,0
coldplay,0.000000,0.000000,0.000000,0.0,0.016712,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0
modest mouse,0.000000,0.000000,0.000000,0.0,0.015935,0.000000,0.0,0.0,0.000000,0.030437,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0


Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [166]:
ratings['cluster'].value_counts()

cluster
0    996
1      1
2      1
4      1
3      1
Name: count, dtype: int64

**Ответ:** Кластеризацию данным алгоритмом нельзя назвать полезной - почти все объекты в одном кластере. Возможно данные имеют сложную струтктуру, с которой алгоритм не справляется.

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [167]:
ratings[ratings['cluster'].isin(ratings['cluster'].value_counts()[ratings['cluster'].value_counts() == 1].index)]

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,cluster
the beatles,0.000000,0.0,0.0,0.0,0.043529,0.0,0.0,0.0,0.093398,0.017621,...,0.0,0.121169,0.038168,0.007939,0.017884,0.0,0.076923,0.0,0.0,1
‌linkin park,0.048932,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,2
the mountain goats,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,4
eno,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.007578,0.000000,0.0,0.000000,0.0,0.0,3


Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Length: 5000, dtype: bool

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --